In [1]:
import os
import csv
import time
import datetime
from tqdm import tqdm
import numpy as np
from itertools import islice

In [2]:
def CSVWriter(outputfile, row):
    csvfile = open(outputfile, 'a+')
    writer = csv.writer(csvfile)
    writer.writerows(row)
    csvfile.close()
    
def getTimeDiff(timeStra, timeStrb):
    if timeStra==timeStrb:
        return 0
    ta = time.strptime(timeStra, "%Y-%m-%d %H:%M:%S")
    tb = time.strptime(timeStrb, "%Y-%m-%d %H:%M:%S")
    y,m,d,H,M,S = ta[0:6]
    dataTimea=datetime.datetime(y,m,d,H,M,S)
    y,m,d,H,M,S = tb[0:6]
    dataTimeb=datetime.datetime(y,m,d,H,M,S)
    secondsDiff=(dataTimeb-dataTimea).seconds
    return secondsDiff
    
def pepa_accerlation(input_file, output_file, timestep, initial_time = '2018-10-25 21:00:00'):
    results = []
    with open(input_file) as fin:
        reader = csv.reader(fin)
        headers = reader.next()
        reader.next()
        reader.next()  # skip the header
        time1 = initial_time
        x = []
        y = []
        z = []
        for record in reader:
            record_dict = dict(zip(headers, record))
            if getTimeDiff(record_dict["time"], time1) < timestep:
                x.append(float(record_dict["pepa_acc_x"]))
                y.append(float(record_dict["pepa_acc_y"]))
                z.append(float(record_dict["pepa_acc_z"]))
                time1 = record_dict["time"]
            elif getTimeDiff(record_dict["time"], time1) >= timestep:
                sub_result = []
                sub_result.append(np.mean(x))
                sub_result.append(np.mean(y))
                sub_result.append(np.mean(z))
                sub_result.append(time1)
                results.append(sub_result)
                del x[:]
                del y[:]
                del z[:]
                x.append(float(record_dict["pepa_acc_x"]))
                y.append(float(record_dict["pepa_acc_y"]))
                z.append(float(record_dict["pepa_acc_z"]))
                time1 = record_dict["time"]
#     CSVWriter(output_file, ['pepa_acc_x', 'pepa_acc_y', 'pepa_acc_z', 'time'])
#     CSVWriter(output_file, ['float', 'float', 'float', 'datetime'])
#     CSVWriter(output_file, ['', '', '', 'T'])
    CSVWriter(output_file, results)

In [3]:
pepa_accerlation('..\\data\\pepa_acc.csv', '..\\data\\pepa_acc_timestep1.csv', 1)